<a href="https://www.kaggle.com/code/karelbecerra/training-enhanced-hand-data?scriptVersionId=212771969" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Prepare notebook environment
> This notebook is for the intention of having an easy start process. You can find more '**production ready**' code for **training** and **inference** into **/classification** repository folder.


Notebook requeriments:
* Install public packages
* Import public packages
* Clone code repository
* Setup working folders
* Link Kaggle dataset to Code repository

In [1]:
# Install public packages
!pip install torchsummary
!pip install munch

# Import public packages
import os
import sys
import numpy as np
from torchsummary import  summary

In [2]:
# Clone repository. After cloning, code will be available in **/kaggle/working/** folder
## Remove local clone if already exist
!rm -rf /kaggle/working/sam-x-ray-medical-images-hand-sex-classification
!git clone https://github.com/karelbecerra/sam-x-ray-medical-images-hand-sex-classification.git

## Move /classification to output root
!mv /kaggle/working/sam-x-ray-medical-images-hand-sex-classification/classification /kaggle/working/classification

## Removing all extra (we need only /classification folder)
!rm -rf /kaggle/working/sam-x-ray-medical-images-hand-sex-classification

Cloning into 'sam-x-ray-medical-images-hand-sex-classification'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 141 (delta 49), reused 90 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (141/141), 8.10 MiB | 35.43 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [3]:
# Setup working folders
troncho_base = '/kaggle/working/classification'
os.chdir(troncho_base)
sys.path.append(troncho_base)

In [4]:
# Link Kaggle dataset to Code repository

# Configs in repository '**/classification/configs**' are expecting to find dataset on '**./datasets**' folders.
# Therefore we need to link Kaggle datasets to ./datasets folder.

# Create a symbolic link
!ln -s /kaggle/input/sam-x-ray-medical-images-hand-sex-classification/enhanced_hand_data ./datasets/enhanced_hand_data

# Training Process
* Import local functions
* Load parameters: Find more in ./configs folders. Parameters include:
  * Augmentations
  * Learning Rate
  * Initial model weights
  * and more...
* Load dataset: setup dataset loaders
* Build model
* Train 

In [5]:
# Import local methods, functions and models (from github repository)
from src.utils import tools
from src.trainer import train_model
from src.loaddataset import load_datasets
from src.buildmodel import build_model
from src.utils import config

In [6]:
# Load parameters
parameters = config.load_parameters(config='enhanced_hand_data', batch=24, epochs=25, run='run1' )

In [7]:
# Load dataset
train_loader, valid_loader, train_dataset_size = load_datasets(parameters=parameters)

In [8]:
# Build model
model, device = build_model(parameters)
summary(model,input_size=(3,400,200))

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 74.9MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 200, 100]             864
       BatchNorm2d-2         [-1, 32, 200, 100]              64
              SiLU-3         [-1, 32, 200, 100]               0
            Conv2d-4         [-1, 32, 200, 100]             288
       BatchNorm2d-5         [-1, 32, 200, 100]              64
              SiLU-6         [-1, 32, 200, 100]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 200, 100]               0
           Conv2d-13         [-1, 16, 200, 100]             512
      BatchNorm2d-14         [-1, 16, 2

In [9]:
# Train model
tools.print_now("START TRAINING")
train_model(parameters=parameters, model=model, device=device, train_loader=train_loader, valid_loader=valid_loader, train_dataset_size=train_dataset_size)
tools.print_now("END TRAINING -- ")

START TRAINING elapsed = 0:00:00  parcial = 0:00:00   now 2024-12-12 22:50:06 
####################################################################################################################
Train - epoch: 0/25 - 521/521 [==============================] - 96.269s -  - loss  0.4947 - 
Validation - 59/59 [==============================] - 3.529s - - loss  0.5405 - 
       #### NEW BEST MODEL ####
Train - epoch: 1/25 - 521/521 [==============================] - 93.020s -  - loss  0.4275 - 
Validation - 59/59 [==============================] - 3.756s - - loss  0.4132 - 
       #### NEW BEST MODEL ####
Train - epoch: 2/25 - 521/521 [==============================] - 92.950s -  - loss  0.2723 - 
Validation - 59/59 [==============================] - 3.832s - - loss  0.3675 - 
       #### NEW BEST MODEL ####
Train - epoch: 3/25 - 521/521 [==============================] - 92.965s -  - loss  0.2890 - 
Validation - 59/59 [==============================] - 3.679s - - loss  0.3452 - 
       #